# Import

In [ ]:
import functools
import itertools
import numpy as np
from scipy.linalg import expm

import qiskit as qk
import qiskit_dynamics as qk_d
import qiskit.providers.fake_provider as qk_fp

import qutip as qt
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import importlib

In [ ]:
# configure jax to use 64 bit mode
import jax
jax.config.update("jax_enable_x64", True)

# tell JAX we are using CPU
jax.config.update('jax_platform_name', 'cpu')

# set default backend
qk_d.array.Array.set_default_backend('jax')
qk_d.array.Array.default_backend()

In [ ]:
import sys
sys.path.append("../")
import pulse_simulator as ps

# Inspect two qubit gate Hamiltonian

In [ ]:
backend = qk_fp.FakeManila()

# Initialize device
# =====
# Undo units
units = 1e9
GHz = 1/units
ns = units

dt = backend.configuration().dt * ns  
duration = 220 * dt  # ns

registers = [0, 1, 2, 3]  # TODO: Active registers

# Variables
# NOTE: If the Rabi rates are different, you have to calibrate!
config_vars = ps.backend_simulation_vars(backend, rabi=False, units=units)

# Carrier frequencies of each control line
carriers = ps.backend_carriers(backend, config_vars)

config_vars

In [ ]:
backend.properties().gate_length('cx', [0, 1])

In [ ]:
# Partially compile to get this circuit's gates
cr_model = functools.partial(
    ps.cross_resonance_model,
    registers=registers,
    backend=backend,
    variables=config_vars, 
    model_name="Simple",
    return_params=True
)

qb_model = functools.partial(
    ps.rx_model,
    registers=registers,
    backend=backend,
    variables=config_vars, 
    rotating_frame=False,
    return_params=True
)

# Control model
for i,j in itertools.permutations(registers, 2):
    if abs(i-j) == 1:
        control = i
        target = j
        H_drift, Hs_control, H_channel, params = cr_model((control, target))
        print(f"Control: {control}, Target: {target}\n Params: {params['CR']}")

print()

# Single qubit model
# Control model
for i in registers:
    H_drift, Hs_control, H_channel, params = qb_model(i)
    print(f"Qubit: {i}, Params: {params}")

In [ ]:
# Partially compile to get this circuit's gates
cr_model = functools.partial(
    ps.cross_resonance_model,
    registers=registers,
    backend=backend,
    variables=config_vars, 
    model_name="SWPT",
    return_params=True
)

qb_model = functools.partial(
    ps.rx_model,
    registers=registers,
    backend=backend,
    variables=config_vars, 
    rotating_frame=False,
    return_params=True
)

# Control model
for i,j in itertools.permutations(registers, 2):
    if abs(i-j) == 1:
        control = i
        target = j
        H_drift, Hs_control, H_channel, params = cr_model((control, target))
        print(f"Control: {control}, Target: {target}\n Params: {params['CR']}")

In [ ]:
-8.61e-03 * -2.76e+00

In [ ]:
# Partially compile to get this circuit's gates
cr_model = functools.partial(
    ps.cross_resonance_model,
    registers=registers,
    backend=backend,
    variables=config_vars, 
    model_name="SWPT",
    return_params=True
)


# Control model
for i,j in itertools.permutations(registers, 2):
    if abs(i-j) == 1:
        control = i
        target = j
        H_drift, Hs_control, H_channel, params = cr_model((control, target))
        print(f"Control: {control}, Target: {target}, Params: {params}")

In [ ]:
def plot_Hamiltonian(H):
    fig, axes = plt.subplots(1, 2)
    kwargs = {"vmin": -.05, "vmax": .05, "cmap": "RdBu"}
    ax = axes[0]
    ax.imshow(np.real(H), **kwargs)
    ax = axes[1]
    ax.imshow(np.imag(H), **kwargs)
    [ax.axis("off") for ax in axes]
    return fig, axes

In [ ]:
# params['IX'], params['ZX']

In [ ]:
plot_Hamiltonian(Hs_control[0])

# ECR study

Notice that if you run two X gates next to each other, the crosstalk condition vanishes.

The key assumptions here are that:
1. The amplitude of the gate should result in an accumulated angle of $\frac{\pi}{2}$. We are using the crosstalk condition for a Hamiltonian that is $H(t) = a(t)X$.
2. The crosstalk condition for a single gate is
\begin{equation}
    \left(\int_0^T \cos\left(2\int_0^t a(s)ds\right)dt \right)^2 + \left(\int_0^T \sin\left(2\int_0^t a(s)ds \right)dt\right)^2
\end{equation}

## Spectators and the gate

\begin{equation}
CS \leftrightarrow C \leftrightarrow T \leftrightarrow TS
\end{equation}

There are two parts to the crosstalk robustness. The first part is due to the X gate. The second part is due to the ZX rotation. They happen sequentially.

First, ignore the ZX rotation. This means studying the impact of the X gate and a control spectator. In this case, the ZZ crosstalk vanishes.

Now, we need to consider how to add the ZX drive back into the calculation. We want to see two things. First, the ZX drive should not affect the ability of the XI drive to cancel crosstalk when both are present. (This is the current behavior of ECR.) Second, the ZX drive should be able to cancel crosstalk on the target spectator. (This would be novel.)

Question:
Why 2at vs. $\int_0^t a dt$

In [ ]:
def angle(t, amp=np.pi/2, duration=1, pulse_duration=1/4):
    wait_duration = (duration - pulse_duration * 2) / 2
    if t < pulse_duration:
        return 0 # amp / pulse_duration
    elif t < pulse_duration + wait_duration:
        return 0.
    elif t < 2 * pulse_duration + wait_duration:
        return amp / pulse_duration
    else:
        return 0.

In [ ]:
duration = 10
pulse = 1.
ts = np.linspace(0, duration, 1000, endpoint=True)
fig, ax = plt.subplots()
ax.plot(ts, [angle(t, duration=duration, pulse_duration=pulse) for t in ts])
ax.set_title("Control vs. time")

In [ ]:
np.trapz([angle(t, amp=np.pi, duration=duration, pulse_duration=pulse) for t in ts], ts)

In the next cell, plot the net crosstalk contributions of sine and cosine. Notice that we get back to zero for both sine and cosine.

In [ ]:
int_angle = np.array([
    np.trapz([
        angle(t, duration=duration, pulse_duration=pulse)
        for t in ts[:m]], ts[:m]
    ) for m in range(len(ts))])
fig, axes = plt.subplots(2,1)
ax = axes[0]
ax.plot(int_angle, label=r"$\int_0^t a(s) ds$")
ax.legend()
ax = axes[1]
cos_vals = [np.trapz(np.cos(2 * int_angle[:m]), ts[:m]) for m in range(len(ts))]
sin_vals = [np.trapz(np.sin(2 * int_angle[:m]), ts[:m]) for m in range(len(ts))]
ax.plot(cos_vals, label="cos")
ax.plot(sin_vals, label="sin")
ax.axhline(pulse, color="black", linestyle="--", alpha=.5, lw=1)
ax.axhline(cos_vals[np.argwhere(ts > pulse)[0][0]], color="black", linestyle="--", alpha=.5, lw=1)
ax.legend()

In [ ]:
(np.trapz([np.cos(2 * i) for i in int_angle], ts)**2, 
np.trapz([np.sin(2 * i) for i in int_angle], ts)**2)
